In [ ]:
import sys 
import numpy as np
import colossus.cosmology.cosmology as Cosmology
import matplotlib.pyplot as plt
from astropy.table import Table

### Import mock data module and setup the configuration 

In [ ]:
import clmm.mock_data as mock

In [ ]:
config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['cosmo'] = 'WMAP7-ML'
config['ngals'] = 10000
config['mdef'] = '200c'

my_data = mock.MockData(config=config)

### Generate the mock catalog from the configuration

In [ ]:
my_data.generate()
my_data.catalog

### Compute and plot shear profile

In [ ]:
from clmm import ShearAzimuthalAverager

In [ ]:
cl_dict = {'z':config['cluster_z'], 'ra':0.0, 'dec': 0.0}
saa = ShearAzimuthalAverager(cl_dict,my_data.catalog)
saa.compute_shear()
shear_prof = saa.make_shear_profile()
saa.plot_profile()
plt.show()